# Standard packages

In [1]:
import os
import pandas as pd
import time

# Specific installs

In [2]:
method_name = "CONTRAFold"

In [ ]:
# Get files in place
!wget http://contra.stanford.edu/contrafold/contrafold_v2_02.tar.gz
!apt-get -q install xz-utils
!tar xf contrafold_v2_02.tar.gz
!rm -f contrafold_v2_02.tar.gz

In [ ]:
os.chdir("/content/contrafold/src")

# Some minor bug-solving
os.system('cat Makefile | awk \'{if($1=="CXXFLAGS"){print $0,"-fpermissive"}else{print $0}}\' > tmp')
os.system('mv tmp Makefile')
os.system('cat Utilities.hpp | awk \'{if($2=="<vector>"){print $0; print "#include <limits.h>"}else{print $0}}\' > tmp')
os.system('mv tmp Utilities.hpp')

!make clean
!make

os.chdir("/content")
exe_path="/content/contrafold/src/contrafold"

# S. cerevisiae (sce) 18 long non-coding RNA dataset
Data source: https://genie.weizmann.ac.il/pubs/PARS10/data/sce_genes_folded.tab.gz

In [5]:
gh_path = "https://raw.githubusercontent.com/sinc-lab/lncRNA-folding/master/data/"
sce = pd.read_csv(gh_path + "sce_genes_folded.tab", delimiter='\t', 
                  header=None, index_col=0, 
                  names=("Gene ID", "sequence", "PARS-assisted folding"))

In [6]:
# Sequences to process
yeast18lnc = ["snR81", "snR34", "snR43", "snR44",  "snR31",  "snR10",
              "snR63", "snR11", "snR82", "snR17b", "snR17a", "snR37",
              "SCR1",  "SRG1",  "snR19", "snR30",  "LSR1",   "TLC1"]

# Compute structures

In [7]:
def run_folding(fasta_name):
    out_file_name = "folded_seq.fasta"
    
    # Set sensitivity related parameter 
    gamma=0.9

    #Run CONTRAfold
    os.system(f"{exe_path} predict {fasta_name} --gamma {gamma} >> tmp")

    # Clean output
    with open(out_file_name, "w") as fout:
        for line in open("tmp"):
            if ">structure" not in line:
                fout.write(line)
    os.remove("tmp")
           
    return out_file_name

In [8]:
out_fasta_name = method_name + "_yeast18"
if os.path.exists(out_fasta_name + ".fasta"): os.remove(out_fasta_name + ".fasta")

lnc_ids = yeast18lnc
print("   \t lnc \t len \t time")
for i, lnc in enumerate(lnc_ids): 

  start_time = time.time()
  seq = sce.loc[lnc]["sequence"]
  print(f"{i+1}/{len(lnc_ids)}\t{lnc} \t {len(seq)}", end='\t')

  # Write a one-sequence fasta
  with open("tmp.fasta", "w") as ofile: 
    ofile.write(f">{lnc}\n{seq}\n")
  
  dot_file_name = run_folding("tmp.fasta")

  # Concatenate outputs
  os.system("cat " + dot_file_name + " >> " + out_fasta_name + ".fasta") 

  os.system('rm -f tmp.fasta')
  print(f"{time.time() - start_time: .1f} s")

   	 lnc 	 len 	 time
1/18	snR81 	 201	 0.2 s
2/18	snR34 	 203	 0.2 s
3/18	snR43 	 209	 0.2 s
4/18	snR44 	 211	 0.2 s
5/18	snR31 	 225	 0.3 s
6/18	snR10 	 245	 0.3 s
7/18	snR63 	 255	 0.4 s
8/18	snR11 	 258	 0.4 s
9/18	snR82 	 268	 0.4 s
10/18	snR17b 	 332	 0.6 s
11/18	snR17a 	 333	 0.6 s
12/18	snR37 	 386	 0.9 s
13/18	SCR1 	 522	 1.9 s
14/18	SRG1 	 551	 2.0 s
15/18	snR19 	 568	 2.4 s
16/18	snR30 	 606	 2.6 s
17/18	LSR1 	 1175	 14.5 s
18/18	TLC1 	 1301	 19.8 s
